In [ ]:
import loader
import importlib
import pandas as pd
importlib.reload(loader)
import importlib
import lexicons
importlib.reload(lexicons)
import text_miner
importlib.reload(text_miner)
import utility
import machine_learning
importlib.reload(machine_learning)


## IGNORE this section and load the json file containing the extracted features in section below under "Load Data with **all** Features"

In [ ]:
load = loader.loader()
corpus = load.corpus
corpus = load.add_ideology_intensity()
#articles_per_ideology_intensity = load.get_article_dfs_per_ideology(ideology='ideology_intensity', include_content=True)


In [ ]:
# The df contains all articles where duplicates were removed. Total is 979 articles.
# It contains the majority effect on liberals and conservatives.
# It contains the ADU counts (other, stats, testimony )
df = pd.read_json('data/articles_with_majority_adus.json', orient='records')
df.set_index('idx', inplace=True)
df.head()

In [ ]:

importlib.reload(lexicons)
df = lexicons.count_nrc_emotions_and_sentiments(df, text_column= 'content')
print(len(df.columns))
df = lexicons.count_mpqa_arg(df, text_column= 'content')
print(len(df.columns))
df = lexicons.count_mpqa_subj_obj(df)
print(len(df.columns))

In [ ]:
liwc_df = loader.loader.load_discourse_liwc()
df = df.join(liwc_df)

## Load Data with Style Features

In [ ]:
#utility.save_json(df.reset_index(), 'articles_with_adu_liwc_lexicons_conntent')
df = pd.read_json('data/articles_with_adu_liwc_lexicons.json', orient='records')
df.set_index('idx', inplace=True)
#df.head()

In [ ]:
import text_miner
def apply_add_lemma(row):
    lemma = ( text_miner.preprocess(row['content']))['lemmas']
    lemma_str = ' '.join(lemma)
    row['lemma'] =lemma_str
    return row

print(df.columns)
df = df.apply(apply_add_lemma, axis=1)
print(df.columns)


In [ ]:
df_train = df[df['split_label'] == 'train']
#importlib.reload(text_miner)
print('1 gram')
df,  extracted_df1=text_miner.extract_n_grams_features(df, df_train, 'lemma', 
                             min_df=0.1, max_df=0.7, ngram_range=(1,1),
                             count_type='tf-idf', idx= 'idx',
                            cols_prefix='lemma1_')

print('2 gram')

df,  extracted_df2=text_miner.extract_n_grams_features(df, df_train, 'lemma', 
                             min_df=0.005, max_df=0.9, ngram_range=(2,2),
                             count_type='tf-idf', idx= 'idx',
                            cols_prefix='lemma2_')

print('3 gram')


In [ ]:
df,  extracted_df3=text_miner.extract_n_grams_features(df, df_train, 'lemma', 
                             min_df=0.005, max_df=0.9, ngram_range=(3,3),
                             count_type='tf-idf', idx= 'idx',
                            cols_prefix='lemma3_')

## Load Data with **all** Features

In [ ]:
#utility.save_json(df.reset_index(), 'articles_with_adu_liwc_lexicons_content')
df = pd.read_json('data/articles_with_adu_liwc_lexicons_content.json', orient='records')
df.set_index('idx', inplace=True)
df.head()

In [ ]:
import time
import acl20_editorial_training
importlib.reload(machine_learning)
importlib.reload(acl20_editorial_training)
def validate_train_test(X_train_df, y_train, X_test_df, y_test, feature_types, validation_folds=5):
    X_train, X_test = acl20_editorial_training.get_instances_with_features_sub(X_train_df, X_test_df, feature_types)

    best_params = machine_learning.svc_param_gridsearch(X_train, y_train, nfolds_or_division=validation_folds)
    

    result =  machine_learning.train_test(X_train, y_train, X_test, y_test, params=best_params)
    result['params'] = best_params
    return result
def run_experiments(df, filename=None, remove_outliers = True, 
                    normalize=True, save_models=False):
    ideologies = ['liberal_majority', 'conservative_majority']
   
    
    r = {}
    
    for ideology in ideologies:
        ## FOR EACH IDEOLOGY
        # For all combination
        print("preprocessing data...")
        X_train_df, y_train, X_test_df, y_test = acl20_editorial_training.get_x_y(df, ideology, 
                                                                                  remove_outliers = True,
                                                                                 normalizing_method="sqrt")
        print("END of preprocessing")

        results = []
        print(ideology)
        print('+++++++++++++++++++++++++++++++++++++++++++++')
        all_feature_types_comb = acl20_editorial_training.get_all_feature_types_comb(df)
        
        for feature_types in all_feature_types_comb:
            result = {}
            
            start_time = time.time()
            result =validate_train_test(X_train_df, y_train, X_test_df, y_test, feature_types, validation_folds=5)
            
            result['features'] = str(feature_types)
            result['ideology'] = ideology
            results.append(result)
            elapsed_time = time.time() - start_time
            print(feature_types, ' ', 'macro-f1: ', result['macro'], 'time(s): ', round(elapsed_time,3))
            print('-------------------------------------------')
            
        r[ideology] = pd.DataFrame.from_dict(results)

            
        if filename is not None:
            r[ideology].sort_values(by=['macro'], ascending=False).to_csv('{}_{}.csv'.format(filename, ideology))
    return r
        

In [ ]:
X_train_df, y_train, X_test_df, y_test = get_x_y(df, 'conservative_majority', remove_outliers = True, normalize=True)
machine_learning.dummy_train_test(X_train_df, y_train, X_test_df, y_test, strategy='uniform')

In [ ]:
importlib.reload(machine_learning)

results= run_experiments(df, filename="models_no-outliers_sqrt")

In [ ]:
def print_stats(df, class_col):
    for split, split_df in df.groupby(class_col):
        print(split)
        print(split_df['split_label'].value_counts()) 

In [ ]:
print_stats(df, 'liberal_majority')

In [ ]:
print_stats(df, 'conservative_majority')

In [ ]:
importlib.reload(machine_learning)
X_train, y_train, X_test, y_test = get_x_y(df, 'liberal_majority', remove_outliers = True, normalize=True)
print()
print(machine_learning.dummy_train_test(X_train.values, y_train, X_test.values, y_test, strategy='stratified'))
print()
print(machine_learning.dummy_train_test(X_train.values, y_train, X_test.values, y_test, strategy='most_frequent'))
print()
print(machine_learning.dummy_train_test(X_train.values, y_train, X_test.values, y_test, strategy='uniform'))

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold

In [ ]:
kf = KFold(n_splits=5)
X = ["a", "b", "c", "d"]
kf = StratifiedKFold(n_splits=5, random_state=1, )
for train, test in kf.split(X):
    print("%s %s" % (train, test))

In [ ]:
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4],
             [1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4]])
y = np.array([0, 0, 1, 1,1,0, 0, 0, 1, 1,1,0])
skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(X, y)

for _, test_index in skf.split(X, y):
    print( "TEST:", test_index)
    X_test = X[test_index]
    
    y_test =y[test_index]

In [ ]:
importlib.reload(acl20_editorial_training)

In [ ]:
import acl20_editorial_training
def save_model(df, feature_types, ideology):
    str_features = '-'.join([str(x) for x in feature_types])
    pkl_filename = 'models/'+ideology+'/'+str_features+'.pkl'

    X_train_df, y_train, X_test_df, y_test = acl20_editorial_training.get_x_y(df, ideology, 
                                                                              remove_outliers = True,
                                                                             normalizing_method="sqrt")
    X_train, X_test = acl20_editorial_training.get_instances_with_features_sub(X_train_df, X_test_df, 
                                                                                       feature_types)
    best_params = machine_learning.svc_param_gridsearch(X_train, y_train, nfolds_or_division=5)
    print('saving file: ', pkl_filename)
    machine_learning.train_save(X_train, y_train, pkl_filename, 
                                                         params=best_params)
    

# Liberal content
save_model(df, [ 'lemma'], 'liberal_majority' )
# Liberal style+content
save_model(df,  ['liwc', 'mpqa_arg', 'mpqa_subjobg', 'lemma'], 'liberal_majority'  )
# Liberal style
save_model(df,['liwc', 'mpqa_subjobg'], 'liberal_majority'  )


In [ ]:
# Liberal content
save_model(df, [ 'lemma'], 'conservative_majority' )
# Liberal style+content
save_model(df,  ['mpqa_arg', 'lemma'], 'conservative_majority'  )
# Liberal style
save_model(df,['nrc', 'adu'], 'conservative_majority'  )

## Significancy between Models

In [ ]:
import time
import acl20_editorial_training
import scipy.stats as stats
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import classification_report, confusion_matrix , accuracy_score, f1_score

import operator
import pickle
def dependent_pairs(dv1, dv2, alpha=0.05):

    # Check if they are normally distriuted
    #diff = list(map(operator.sub, dv1, dv2))
    is_normal = stats.shapiro(diff)[1] > 0.05
    
    stat, p_val = stats.ttest_rel(dv1,dv2) if is_normal else stats.wilcoxon(dv1,dv2)

    return stat, p_val, is_normal

def run_experiments_with_test_repetition(df, all_feature_types_comb, ideology, score='macro'):
    ideologies = ['liberal_majority', 'conservative_majority']
    
    result = {}
    r=[]
    ## FOR EACH IDEOLOGY
    # For all combination#
    
    print("preprocessing data...")
    X_train_df, y_train, X_test_df, y_test = acl20_editorial_training.get_x_y(df, ideology, 
                                                                              remove_outliers = True,
                                                                             normalizing_method="sqrt")
    print("END of preprocessing")

    
    
    print(ideology)
    print('+++++++++++++++++++++++++++++++++++++++++++++')
    #all_feature_types_comb = get_all_feature_types_comb(df)

    for feature_types in all_feature_types_comb:
        
        print(feature_types)

        start_time = time.time()
        if feature_types[0] == 'dummy':
            f = all_feature_types_comb[1]
            X_train, X_test = acl20_editorial_training.get_instances_with_features_sub(X_train_df, X_test_df, 
                                                                                       f)

        else:
            X_train, X_test = acl20_editorial_training.get_instances_with_features_sub(X_train_df, X_test_df, 
                                                                                       feature_types)
            

        ## 1. svm search grid - Leave one out validation
        ## get best param the test on test set 10 folds
        #print(best_params)
        elapsed_time = time.time() - start_time


            
        skf = StratifiedKFold(n_splits=5)
        skf.get_n_splits(X_test, y_test)

        
        runs = []
        for _, test_index in skf.split(X_test, y_test):
            X_sub_test = X_test[test_index]

            y_sub_test =y_test[test_index]
            
            if len(X_sub_test) != 0:
                if feature_types[0] == 'dummy':
                    macro =  machine_learning.dummy_train_test(X_train, y_train, X_sub_test, y_sub_test, 
                                                               strategy='uniform')[score]
                else:
                    str_features = '-'.join([str(x) for x in feature_types])
                    pkl_filename = 'models/'+ideology+'/'+str_features+'.pkl'
                    
                    with open(pkl_filename, 'rb') as file:  
                        #print('loading model: ' + pkl_filename)
                        pickle_model = pickle.load(file)
                        y_pred = pickle_model.predict(X_sub_test) 
                        macro = f1_score(y_pred=y_pred, y_true=y_sub_test, average=score)
                    
                    
                runs.append(macro)
                elapsed_time = time.time() - start_time
   
        result[str(feature_types)] = runs
        r.append(runs)
        #print(n, ' - ', feature_types, ' ', results)
            
    significancy = dependent_pairs(r[0], r[1]) 
                #r[ideology] = pd.DataFrame.from_dict(results)
        #if filename is not None:
        #    r[ideology].sort_values(by=['macro'], ascending=False).to_csv('{}_{}.csv'.format(filename, ideology))
    return significancy, result
        

In [ ]:
liberal_content_baseline[0][1] < 0.05

In [ ]:
print('liberal_content_baseline: ')
print(liberal_content_baseline_micro )

print('\nliberal_content_contentstyle: ')
print(liberal_content_contentstyle_micro)
print('\nliberal_contentstyle_baseline: ')
print(liberal_contentstyle_baseline_micro )
print('\nliberal_style_baseline: ')
print(liberal_style_baseline_micro)
print('\nliberal_style_content: ')
print(liberal_style_content_micro)
print('\nliberal_style_stylecontent: ')
print(liberal_style_stylecontent_micro )

In [ ]:
print('liberal_content_baseline: ')
print(liberal_content_baseline_micro[0][1] < 0.05)

print('\nliberal_content_contentstyle: ')
print(liberal_content_contentstyle_micro[0][1] < 0.05)
print('\nliberal_contentstyle_baseline: ')
print(liberal_contentstyle_baseline_micro[0][1] < 0.05)
print('\nliberal_style_baseline: ')
print(liberal_style_baseline_micro[0][1] < 0.05)
print('\nliberal_style_content: ')
print(liberal_style_content_micro[0][1] < 0.05)
print('\nliberal_style_stylecontent: ')
print(liberal_style_stylecontent_micro[0][1] < 0.05)

In [ ]:
print('liberal_content_baseline: ')
print(liberal_content_baseline[0][1] < 0.05)

print('\nliberal_content_contentstyle: ')
print(liberal_content_contentstyle[0][1] < 0.05)
print('\nliberal_contentstyle_baseline: ')
print(liberal_contentstyle_baseline[0][1] < 0.05)
print('\nliberal_style_baseline: ')
print(liberal_style_baseline[0][1] < 0.05)
print('\nliberal_style_content: ')
print(liberal_style_content[0][1] < 0.05)
print('\nliberal_style_stylecontent: ')
print(liberal_style_stylecontent[0][1] < 0.05)

print('liberal_content_baseline: ')
print(liberal_content_baseline[0][1] )

print('\nliberal_content_contentstyle: ')
print(liberal_content_contentstyle[0][1] )
print('\nliberal_contentstyle_baseline: ')
print(liberal_contentstyle_baseline[0][1] )
print('\nliberal_style_baseline: ')
print(liberal_style_baseline[0][1] )
print('\nliberal_style_content: ')
print(liberal_style_content[0][1] )
print('\nliberal_style_stylecontent: ')
print(liberal_style_stylecontent[0][1])

In [ ]:
# LIBERAL
# style + content VS. Content
print('Liberal')

# (content) VS. dummy  NOT SIGNIFICANT
liberal_content_baseline_micro = run_experiments_with_test_repetition(df, [['dummy'],['lemma']],
                                     'liberal_majority', score='micro')
    
    
liberal_content_contentstyle_micro = run_experiments_with_test_repetition(df, [['liwc', 'mpqa_arg', 'mpqa_subjobg', 'lemma'],
                                                                        ['lemma']],
                                     'liberal_majority', score='micro')


liberal_contentstyle_baseline_micro = run_experiments_with_test_repetition(df, [['dummy'], ['liwc', 'mpqa_arg',
                                                                                      'mpqa_subjobg', 'lemma']],
                                     'liberal_majority', score='micro')

liberal_style_baseline_micro = run_experiments_with_test_repetition(df, [['dummy'], ['liwc', 'mpqa_subjobg'] 
                                                                     ],
                                     'liberal_majority', score='micro')

liberal_style_content_micro = run_experiments_with_test_repetition(df, [['lemma'], ['liwc', 'mpqa_subjobg'] 
                                                                     ],
                                     'liberal_majority', score='micro')

liberal_style_stylecontent_micro = run_experiments_with_test_repetition(df, [['liwc', 'mpqa_arg', 'mpqa_subjobg', 'lemma'], ['liwc', 'mpqa_subjobg'] 
                                                                     ],
                                     'liberal_majority', score='micro')


In [ ]:
# LIBERAL
# style + content VS. Content
print('Conservative') # (1.0, 0.0782481864330958), NOT SIGNI
print("(style + content) VS. Content")
cons_contentstyle_content_strat = run_experiments_with_test_repetition(df, [['mpqa_arg', 'lemma'], ['lemma']],
                                     'conservative_majority', score='macro')



print("(style + content) VS. dummy") # baseline SIGN better: 0.0, 0.043114446783075355
cons_contentstyle_dummy_strat = run_experiments_with_test_repetition(df, [['dummy'], ['mpqa_arg', 'lemma']],
                                     'conservative_majority', score='macro')


print("content VS. dummy") # baseline SIGN better: (0.0, 0.04216819709715596)
cons_content_dummy_strat = run_experiments_with_test_repetition(df, [['dummy'], ['lemma']],
                                     'conservative_majority', score='macro')
    
    


In [ ]:

cons_content_style = ['mpqa_arg', 'lemma']
cons_style = ['nrc', 'adu']
conservative_content_baseline = run_experiments_with_test_repetition(df, [['dummy'],['lemma']],
                                     'conservative_majority', score='macro')
    
    
conservative_content_contentstyle = run_experiments_with_test_repetition(df, [cons_content_style,
                                                                        ['lemma']],
                                     'conservative_majority', score='macro')


conservative_contentstyle_baseline = run_experiments_with_test_repetition(df, [['dummy'], cons_content_style],
                                     'conservative_majority', score='macro')

conservative_style_baseline = run_experiments_with_test_repetition(df, [['dummy'],  cons_style  ],
                                     'conservative_majority', score='macro')

conservative_style_content = run_experiments_with_test_repetition(df, [['lemma'],  cons_style ],
                                     'conservative_majority', score='macro')

conservative_style_stylecontent = run_experiments_with_test_repetition(df, [cons_style, cons_content_style 
                                                                     ],
                                     'conservative_majority', score='macro')

In [ ]:
print('conservative_content_baseline: ')
print(conservative_content_baseline[0][1] )

print('\nconservative_content_contentstyle: ')
print(conservative_content_contentstyle[0][1] )
print('\nconservative_contentstyle_baseline: ')
print(conservative_contentstyle_baseline[0][1] )
print('\nconservative_style_baseline: ')
print(conservative_style_baseline[0][1] )
print('\nconservative_style_content: ')
print(conservative_style_content[0][1] )
print('\nconservative_style_stylecontent: ')
print(conservative_style_stylecontent[0][1] )



MACRO

liberal - content VS. baseline: 
False
liberal - contentstyle VS. baseline: 
True
liberal - style VS. baseline: 
True



liberal - style VS. content: 
False
liberal - content VS. contentstyle: 
True
liberal - style VS. stylecontent: 
False